## 🤖 Solution: A PPO-Powered Trading Agent

This notebook implements a complete Deep Reinforcement Learning (DRL) pipeline to train an autonomous trading agent. The goal is to develop a policy that outperforms the market by making intelligent decisions on when to go long, short, or stay neutral.

The solution is structured as follows:
1.  **Environment Setup & Data Preparation:** We install libraries, load the data, engineer 12 distinct market features, and split the data into chronological `train`, `validation`, and `test` sets.
2.  **Baseline Agent:** We establish a "Random Agent" baseline to measure our agent's effectiveness.
3.  **PPO Agent Implementation:** We build our agent from scratch using Proximal Policy Optimization (PPO) with an Actor-Critic network.
4.  **Hyperparameter Tuning:** We use `Optuna` to automatically find the best set of hyperparameters (learning rate, network size, etc.) by evaluating models on the `validation` set.
5.  **Final Model Training:** We train the agent with the *best* hyperparameters on the *full* training dataset (`df_train_full`).
6.  **Final Evaluation & Visualization:** We load the best saved model and run it on the *unseen* `test` set (`df_eval`) to get our final project result.

---

## 1. 🛠️ Environment Setup & Dependencies

Before we can build our agent, we must set up the environment. This involves two steps:

* **Installing Packages:** We use `pip` to install the core libraries:
    * `gym-trading-env`: The trading simulation environment.
    * `torch`: The deep learning framework for our agent's neural network.
    * `optuna`: For hyperparameter optimization.
* **Importing Libraries:** We import all the necessary tools for data manipulation (`pandas`, `numpy`), environment creation (`gym`), and agent building (`torch.nn`).

In [ ]:
!pip3 install -r requirements.txt --quiet

In [145]:
import numpy as np
import pandas as pd
import gymnasium as gym
import gym_trading_env
from gym_trading_env.downloader import download
from pathlib import Path
import matplotlib.pyplot as plt
import time
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch
import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler


## 2. 📈 Data Preprocessing & Feature Engineering

# Load the dataset

In [146]:

# --- Setup Folders ---
data_folder = Path("data/")
data_folder.mkdir(parents=True, exist_ok=True)
eval_folder = Path("eval/")
eval_folder.mkdir(parents=True, exist_ok=True)

"""
download(exchange_names = ["binance"],
    symbols= ["BTC/USDT"],
    timeframe= "1h",
    dir = data_folder,
    since= datetime.datetime(year= 2020, month=10, day=1),
)"""

# 1. Load Data
df = pd.read_pickle(data_folder / "binance-BTCUSDT-1h.pkl")



# Exploration of dataset

In [147]:
df.head()

,open,high,low,close,volume,date_close
date_open,,,,,,
2020-09-30 23:00:00,10745.85,10785.00,10735.51,10776.59,1235.545956,2020-10-01 00:00:00
2020-10-01 00:00:00,10776.59,10826.19,10776.59,10788.06,2128.759531,2020-10-01 01:00:00
2020-10-01 01:00:00,10788.30,10849.97,10786.74,10838.88,1604.129560,2020-10-01 02:00:00
2020-10-01 02:00:00,10838.89,10857.47,10807.39,10817.14,1268.291734,2020-10-01 03:00:00
2020-10-01 03:00:00,10817.14,10824.22,10789.01,10798.18,939.599057,2020-10-01 04:00:00


The agent needs to asses at each begining of the hours if we should buy, sell or whatever. So It should not get the close value of that hour as we do not know it. This is a "Look-Ahead Bias". When We think about it, many features have a look-Ahead bias (`high`,`low`)

PS : We can see that the `close` of a previous hour is the `open` of the next hour. As such, to prevent the agent to get the open value of the next hour we will shift the value to get `previous_close`

# Creating features for the agent

In [148]:
# --- 1. Initial Data Shift and Cleanup ---
# Shift 'close' one step back to create 'prev_close'. This is the price
# available at the moment the new candle OPENS (i.e., Close at t-1).
df["prev_close"] = df["close"].shift(1)

# --- 2. Calculate Raw Technical Indicators on LAGGED DATA ---
# All indicators MUST use 'prev_close' for their core calculations.

# Sharpe Ratio
ANNUALIZATION_FACTOR = 24 * 365
ROLLING_WINDOW_SR = 7 * 24 
RISK_FREE_RATE_ANNUAL = 0.04
RISK_FREE_RATE_HOURLY = (1 + RISK_FREE_RATE_ANNUAL)**(1/ANNUALIZATION_FACTOR) - 1

# Base returns calculation uses 'prev_close' (i.e., Close at t-1)
df['return'] = df['prev_close'].pct_change()
df['excess_return'] = df['return'] - RISK_FREE_RATE_HOURLY
rolling_mean_excess = df['excess_return'].rolling(window=ROLLING_WINDOW_SR).mean()
rolling_std_excess = df['excess_return'].rolling(window=ROLLING_WINDOW_SR).std()
df['raw_sharpe'] = (rolling_mean_excess / (rolling_std_excess + 1e-9)) * np.sqrt(ANNUALIZATION_FACTOR)

# MACD uses 'prev_close' for EMAs
df['EMA_12'] = df['prev_close'].ewm(span=12, adjust=False).mean()
df['EMA_26'] = df['prev_close'].ewm(span=26, adjust=False).mean()
df['raw_macd'] = df['EMA_12'] - df['EMA_26']
df['raw_macd_signal'] = df['raw_macd'].ewm(span=9, adjust=False).mean()

# Bollinger Bands uses 'prev_close' for MA and StdDev
ROLLING_WINDOW_BB = 20
df['BB_Middle'] = df['prev_close'].rolling(window=ROLLING_WINDOW_BB).mean()
df['BB_Std'] = df['prev_close'].rolling(window=ROLLING_WINDOW_BB).std()
df['raw_bb_upper'] = df['BB_Middle'] + (df['BB_Std'] * 2)
df['raw_bb_lower'] = df['BB_Middle'] - (df['BB_Std'] * 2)

# OBV uses 'prev_close'
df['raw_obv'] = (np.sign(df['prev_close'].diff()) * df['volume'].shift(1)).cumsum().fillna(0)


# ATR (Average True Range) - NEW
df['high_t_minus_1'] = df['high'].shift(1)
df['low_t_minus_1'] = df['low'].shift(1)
df['prev_prev_close'] = df['prev_close'].shift(1)

df['tr_1'] = df['high_t_minus_1'] - df['low_t_minus_1'] # Range of candle t-1
df['tr_2'] = np.abs(df['high_t_minus_1'] - df['prev_prev_close']) # Distance from previous close to high
df['tr_3'] = np.abs(df['low_t_minus_1'] - df['prev_prev_close']) # Distance from previous close to low
df['true_range'] = df[['tr_1', 'tr_2', 'tr_3']].max(axis=1)
df['raw_atr'] = df['true_range'].rolling(window=14).mean()

# RSI (Relative Strength Index) - NEW
delta = df['prev_close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / (loss + 1e-9)
df['raw_rsi'] = 100 - (100 / (1 + rs))

# --- 3. Add Cyclical Time Features (No Change, Already Safe) ---
df['hour'] = df.index.hour
df['day_of_week'] = df.index.dayofweek

df['feature_hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
df['feature_hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)
df['feature_day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['feature_day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

# --- 4. Create Final, Normalized Features (Shift Applied When Necessary) ---

# We define a log-return feature based on data available in the previous completed candle.
df['feature_log_return_1h'] = np.log(df['prev_close'] / df['prev_close'].shift(1))


# Price Features (Normalized by prev_close, ready for t=0 observation)
df['feature_open'] = (df['open'] / df['prev_close']) - 1
df['feature_high'] = (df['high'].shift(1) / df['prev_close']) - 1
df['feature_low'] = (df['low'].shift(1) / df['prev_close']) - 1


# Volume Features (Z-Score of volume/OBV at t-1)
vol_mean_30d = df['volume'].shift(1).rolling(30*24).mean()
vol_std_30d = df['volume'].shift(1).rolling(30*24).std()
df['feature_volume_zscore'] = ((df['volume'].shift(1) - vol_mean_30d) / (vol_std_30d + 1e-9))
obv_mean_30d = df['raw_obv'].shift(1).rolling(30*24).mean()
obv_std_30d = df['raw_obv'].shift(1).rolling(30*24).std()
df['feature_obv_zscore'] = ((df['raw_obv'].shift(1) - obv_mean_30d) / (obv_std_30d + 1e-9))

# Indicator Features
df['feature_MACD'] = (df['raw_macd'].shift(1) / df['prev_close'])
df['feature_MACD_Signal'] = (df['raw_macd_signal'].shift(1) / df['prev_close'])
df['feature_BB_Upper'] = (df['raw_bb_upper'].shift(1) / df['prev_close']) - 1
df['feature_BB_Lower'] = (df['raw_bb_lower'].shift(1) / df['prev_close']) - 1
df['feature_atr'] = (df['raw_atr'].shift(1) / df['prev_close'])
df['feature_rsi'] = df['raw_rsi'].shift(1)
df['feature_sharpe_ratio'] = df['raw_sharpe'].shift(1)

# --- 5. Final Cleanup ---
final_features = [
    'feature_hour_sin', 'feature_hour_cos', 'feature_day_sin', 'feature_day_cos',
    'feature_open', 'feature_high', 'feature_low', 'feature_log_return_1h',
    'feature_volume_zscore', 'feature_obv_zscore',
    'feature_MACD', 'feature_MACD_Signal', 
    'feature_BB_Upper', 'feature_BB_Lower',
    'feature_atr', 'feature_rsi', 'feature_sharpe_ratio'
]

# Keep the current raw OHLCV for the Environment to calculate rewards/penalties,
# but the agent MUST only observe the 'feature_' columns.
all_cols_to_keep = ['close','open', 'high', 'low', 'prev_close', 'volume'] + final_features
df = df[all_cols_to_keep]

df.dropna(inplace=True)

# --- 6. Defining DataFrames ---
# (Environment setup remains correct as it depends on the index slicing)
df_eval = df.loc['2025-10-01':'2025-11-01'] 
df_train_full = df.loc['2024-10-01':'2025-09-30'] 
df_train = df_train_full.loc['2024-10-01':'2025-06-30']
df_validation = df_train_full.loc['2025-07-01':'2025-09-30']

# Reward function 
In previous attemps, we noticed that the agent has a tendencie to do nothing. As Such, we will add a Neutrality penality as to make sure the agent does not do anything. 

In [149]:
def custom_reward(historical_info: dict):
    # Position: The position held *during* the last completed step (t).
    position = historical_info["position", -1] 
    
    # Prices: Use the Close price of the completed bar (t) and the Close price of the bar before it (t-1).
    # This represents the fractional return of the asset over the last hour.
    current_close = historical_info["data_close", -1] 
    previous_close = historical_info["data_close", -2] 
    
    # 1. Calculate the asset's fractional return during the step
    # Note: Using .pct_change() logic is generally more stable than (C-P)/P
    # return is the return of bar t.
    asset_return = (current_close - previous_close) / previous_close
    
    # 2. Calculate Portfolio PnL for this step
    # PnL = (Asset Return) * (Held Position)
    pnl = asset_return * position

    # 3. Define and Apply the Neutrality Penalty
    NEUTRAL_PENALTY = -0.000001
    
    reward = pnl
    if position == 0:
        # Penalize for holding cash (or no position)
        reward += NEUTRAL_PENALTY
        
    return reward

In [150]:
# --- Creation of Three Environments ---

POSITIONS = [-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1]
WINDOW_SIZE = 48
TRADING_FEES =  0.01/100
BORROW_INTEREST_RATE = 0.0003/100

# Environment for TRAINING
env_train = gym.make("TradingEnv",
        name= "BTCUSD_Train",
        df = df_train, 
        windows=WINDOW_SIZE,
        positions = POSITIONS,
        trading_fees = TRADING_FEES, 
        borrow_interest_rate= BORROW_INTEREST_RATE,
        reward_function=custom_reward
    )

# Environment for VALIDATION (used inside Optuna)
env_validation = gym.make("TradingEnv",
        name= "BTCUSD_Validation",
        df = df_validation,
        windows=WINDOW_SIZE,
        positions = POSITIONS,
        trading_fees = TRADING_FEES, 
        borrow_interest_rate= BORROW_INTEREST_RATE,
        reward_function=custom_reward
    )

# Environment for FINAL TEST (used only once)
env_eval = gym.make("TradingEnv",
        name= "BTCUSD_Eval",
        df = df_eval, 
        windows=WINDOW_SIZE,
        positions = POSITIONS,
        trading_fees = TRADING_FEES, 
        borrow_interest_rate= BORROW_INTEREST_RATE,
        reward_function=custom_reward

    )

In [151]:
def evaluate_agent(agent, env, num_episodes=20, max_steps=None, render=False, csv_path="evaluation_results.csv", renderer_logs_dir="render_logs"):
    """
    Evaluate the agent on the environment for a number of episodes.
    """
    results = []
    
    # Ensure render dir exists
    if render:
        Path(renderer_logs_dir).mkdir(parents=True, exist_ok=True)

    for ep in range(num_episodes):
        obs, info = env.reset()
        done = False
        truncated = False
        step = 0
        reward_total = 0.0
        while not done and not truncated:
            action = agent.choose_action_eval(obs)
            obs, reward, done, truncated, info = env.step(action)
            reward_total += reward
            step += 1
            if (max_steps is not None) and (step >= max_steps):
                break

        metrics = env.get_metrics()
        port_ret = float(metrics["Portfolio Return"].strip('%')) / 100.0
        market_ret = float(metrics["Market Return"].strip('%')) / 100.0

        results.append({
            "episode": ep + 1,
            "portfolio_return": port_ret,
            "market_return": market_ret,
            "excess_return": port_ret - market_ret,
            "steps": step,
            "total_reward": reward_total,
        })
        
        if render:
            print(f"Eval Episode {ep+1}: Total Reward: {reward_total:.2f}, Portfolio Return: {port_ret:.2%}, Market Return: {market_ret:.2%}, Excess Return: {(port_ret - market_ret):.2%}, Steps: {step}")
            time.sleep(1)
            env.save_for_render(dir=renderer_logs_dir)

    df_results = pd.DataFrame(results)
    
    # Ensure the directory for the CSV exists
    Path(csv_path).parent.mkdir(parents=True, exist_ok=True)
    df_results.to_csv(csv_path, index=False)
    print(f"Saved evaluation results to {csv_path}")

    return df_results

## 3. 🎲 Baseline: The Random Agent

Before we build a complex DRL agent, we must establish a baseline. If our "smart" agent can't beat an agent that takes random actions, it has learned nothing.

The `RandomAgent` simply chooses a random action (a position from -1 to 1) from the environment's action space at every step. We will evaluate this agent on the **test set** to see what score we need to beat.

In [152]:
class RandomAgent:
    def __init__(self, action_space):
        self.action_space = action_space

    def act(self, observation):
        return self.action_space.sample()

    def choose_action_eval(self, state):
        return self.action_space.sample()

In [153]:
# Create a random agent for evaluation
agent = RandomAgent(env_eval.action_space)

# Evaluate the trained agent
df_results = evaluate_agent(agent, env_eval, num_episodes=20, render=True, csv_path=eval_folder / "evaluation_results.csv", renderer_logs_dir=eval_folder / "render_logs")

Market Return : -8.65%   |   Portfolio Return : -6.70%   |   
Eval Episode 1: Total Reward: -0.01, Portfolio Return: -6.70%, Market Return: -8.65%, Excess Return: 1.95%, Steps: 720
Market Return : -8.65%   |   Portfolio Return : -1.26%   |   
Eval Episode 2: Total Reward: 0.05, Portfolio Return: -1.26%, Market Return: -8.65%, Excess Return: 7.39%, Steps: 720
Market Return : -8.65%   |   Portfolio Return : -1.99%   |   
Eval Episode 3: Total Reward: 0.04, Portfolio Return: -1.99%, Market Return: -8.65%, Excess Return: 6.66%, Steps: 720
Market Return : -8.65%   |   Portfolio Return : -19.38%   |   
Eval Episode 4: Total Reward: -0.16, Portfolio Return: -19.38%, Market Return: -8.65%, Excess Return: -10.73%, Steps: 720
Market Return : -8.65%   |   Portfolio Return : -13.92%   |   
Eval Episode 5: Total Reward: -0.09, Portfolio Return: -13.92%, Market Return: -8.65%, Excess Return: -5.27%, Steps: 720
Market Return : -8.65%   |   Portfolio Return : -12.79%   |   
Eval Episode 6: Total Rewar

## 4. 🧠 Building the PPO Agent

This is the core of our project. We are implementing a **Proximal Policy Optimization (PPO)** agent from scratch using PyTorch.

### Why PPO?
PPO is a robust, state-of-the-art algorithm that balances exploration (trying new things) and exploitation (using what works). It uses a "clipped" objective function to prevent updates that are too large, which leads to more stable and reliable training than older methods.

### Architecture
Our PPO agent consists of two key components:

1.  **`ActorCriticNetwork`:** This single neural network serves two purposes:
    * **Actor (Policy):** It decides *what action to take* (e.g., "go long", "go short"). It outputs a probability distribution over all 9 possible actions.
    * **Critic (Value):** It estimates *how good the current state is* (i.e., the expected future reward). It outputs a single value, which helps the Actor learn better.

2.  **`PPOAgent`:** This class manages the entire learning process. It:
    * Holds the `ActorCriticNetwork` and its optimizer.
    * Gathers experience from the environment (`store`).
    * Calculates advantages using Generalized Advantage Estimation (GAE) to determine how much better an action was than expected (`compute_gae`).
    * Runs the PPO update logic across multiple epochs to improve the network (`update`).

In [154]:

class ActorCriticNetwork(nn.Module):
    def __init__(self, state_shape, n_actions, hidden_size=128, n_layers=2):
        super().__init__()
        self.window_size = state_shape[0]
        self.n_features = state_shape[1]

        # --- 1. CNN Layers ---
        # Input: (Batch, Features, Window)
        self.cnn_layers = nn.Sequential(
            nn.Conv1d(in_channels=self.n_features, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU()
        )

        # The convolution output size is (Batch, 64, self.window_size).
        self.flattened_size = 64 * self.window_size 

        # --- 2. Dynamic Shared Linear Layers (Fixing n_layers) ---
        layers = []
        input_dim = self.flattened_size
        
        for _ in range(n_layers):
            layers.append(nn.Linear(input_dim, hidden_size))
            layers.append(nn.ReLU())
            input_dim = hidden_size # Next layer takes hidden_size as input
            
        self.shared_linear = nn.Sequential(*layers)

        # --- 3. Heads ---
        self.actor = nn.Linear(hidden_size, n_actions)
        self.critic = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # x shape: (Batch, Window, Features)
        
        # Permute for Conv1d: (Batch, Features, Window)
        x_permuted = x.permute(0, 2, 1) 
        
        cnn_out = self.cnn_layers(x_permuted)
        x_flat = cnn_out.view(-1, self.flattened_size)
        
        features = self.shared_linear(x_flat)

        # --- Fix: Return Logits (Raw scores), NOT Softmax ---
        action_logits = self.actor(features) 
        state_value = self.critic(features)

        return action_logits, state_value
    

class PPOAgent:
    def __init__(
        self, state_size, n_actions,
        lr=3e-4, gamma=0.99, gae_lambda=0.95,
        entropy_beta=0.01, clip_epsilon=0.2, ppo_epochs=10, batch_size=64,
        hidden_size=128,
        n_layers=2  # <<< --- 1. ADD THIS (with a default)
    ):
        
        # Hyperparameters
        self.lr = lr
        self.gamma = gamma
        self.gae_lambda = gae_lambda
        self.entropy_beta = entropy_beta
        self.clip_epsilon = clip_epsilon
        self.ppo_epochs = ppo_epochs
        self.batch_size = batch_size

        # Environment parameters
        self.state_size = state_size
        self.n_actions = n_actions

        # Device configuration
        if torch.backends.mps.is_available():
            self.device = torch.device("mps")  
        else:
            self.device = torch.device("cpu")

        # Create policy network
        self.network = ActorCriticNetwork(
            state_size, 
            n_actions, 
            hidden_size, 
            n_layers  
        ).to(self.device)

        # Optimizer
        self.optimizer = optim.Adam(self.network.parameters(), lr=lr)

        # Memory buffers
        self.reset_memory()

    def reset_memory(self):
        """Clear rollout buffers."""
        self.states = []
        self.actions = []
        self.rewards = []
        self.values = []
        self.dones = []
        self.log_probs = []

    def get_action_value_logprob(self, state):
        """
        Samples an action for the training loop.
        Returns the action, its value, and log probability.
        """
        state_tensor = torch.tensor(state, dtype=torch.float32, device=self.device).unsqueeze(0)

        with torch.no_grad():
            logits, value = self.network(state_tensor)

        probs = F.softmax(logits, dim=-1)
        dist = torch.distributions.Categorical(probs=probs)
        action = dist.sample()
        log_prob = dist.log_prob(action)

        return action.item(), value.item(), log_prob.item()

    def choose_action_eval(self, state):
        """
        Chooses the best action for evaluation (deterministic).
        Returns only the action index.
        """
        state_tensor = torch.tensor(state, dtype=torch.float32, device=self.device).unsqueeze(0)
        
        with torch.no_grad():
            logits, _ = self.network(state_tensor)
        
        probs = F.softmax(logits, dim=-1)
        action = torch.argmax(probs, dim=-1)
        
        return action.item()

    def store(self, state, action, reward, value, done, log_prob):
        """Store a single transition in memory."""
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.values.append(value)
        self.dones.append(done)
        self.log_probs.append(log_prob)

    def compute_gae(self, next_value):
        """
        Compute returns and advantages using GAE (Generalized Advantage Estimation)
        """
        rewards = np.array(self.rewards, dtype=np.float32)
        values = np.array(self.values + [next_value], dtype=np.float32)
        dones = np.array(self.dones, dtype=np.float32)

        T = len(rewards)
        returns = np.zeros(T, dtype=np.float32)
        advantages = np.zeros(T, dtype=np.float32)

        gae = 0.0
        for t in reversed(range(T)):
            delta = rewards[t] + self.gamma * values[t + 1] * (1.0 - dones[t]) - values[t]
            gae = delta + self.gamma * self.gae_lambda * (1.0 - dones[t]) * gae
            advantages[t] = gae
            returns[t] = advantages[t] + values[t]

        return returns, advantages

    def update(self, next_value):
        """Perform one PPO update step."""
        if len(self.states) == 0:
            return {"actor_loss": 0.0, "critic_loss": 0.0}

        returns, advantages = self.compute_gae(next_value)

        # Convert to tensors
        states = torch.tensor(np.array(self.states), dtype=torch.float32, device=self.device)
        actions = torch.tensor(np.array(self.actions), dtype=torch.int64, device=self.device)
        returns = torch.tensor(returns, dtype=torch.float32, device=self.device)
        advantages = torch.tensor(advantages, dtype=torch.float32, device=self.device)
        old_log_probs = torch.tensor(np.array(self.log_probs), dtype=torch.float32, device=self.device)

        # Normalize advantages
        advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

        total_actor_loss = 0
        total_critic_loss = 0
        updates = 0
        
        for _ in range(self.ppo_epochs):
            indices = torch.randperm(len(states))
            
            for start in range(0, len(states), self.batch_size):
                end = start + self.batch_size
                idx = indices[start:end]
                
                if len(idx) == 0:
                    continue

                batch_states = states[idx]
                batch_actions = actions[idx]
                batch_old_log_probs = old_log_probs[idx]
                batch_returns = returns[idx]
                batch_advantages = advantages[idx]
                
                # Forward pass
                logits, values = self.network(batch_states)
                action_probs = F.softmax(logits, dim=-1)
                dist = torch.distributions.Categorical(action_probs)
                log_probs = dist.log_prob(batch_actions)
                entropy = dist.entropy().mean()
                
                # PPO loss computation
                ratio = torch.exp(log_probs - batch_old_log_probs)
                surr1 = ratio * batch_advantages
                surr2 = torch.clamp(ratio, 1 - self.clip_epsilon, 1 + self.clip_epsilon) * batch_advantages
                actor_loss = -torch.min(surr1, surr2).mean()
                
                critic_loss = (batch_returns - values.squeeze()).pow(2).mean()
                
                loss = actor_loss + 0.5 * critic_loss - self.entropy_beta * entropy
                
                self.optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.network.parameters(), 0.5)
                self.optimizer.step()
                
                total_actor_loss += actor_loss.item()
                total_critic_loss += critic_loss.item()
                updates += 1
        
        self.reset_memory()
        
        if updates == 0:
            return {"actor_loss": 0.0, "critic_loss": 0.0}

        return {
            'actor_loss': total_actor_loss / updates,
            'critic_loss': total_critic_loss / updates
        }

In [155]:
# Get state and action dimensions from the environment
state_shape = env_eval.observation_space.shape  # This will be (10, 12)
n_actions = env_eval.action_space.n

print(f"State shape: {state_shape}")
print(f"Number of actions: {n_actions}")

# Create the agent with the correct dimensions
agent = PPOAgent(
    state_size=state_shape, 
    n_actions=n_actions,
    n_layers=2 # Explicitly using the new param we added
)
# Evaluate the (untrained) agent
# This will now run without errors
df_results = evaluate_agent(agent, env_eval, num_episodes=10, render=True, csv_path=eval_folder / "evaluation_results.csv", renderer_logs_dir=eval_folder / "render_logs")

State shape: (48, 19)
Number of actions: 9
Market Return : -8.65%   |   Portfolio Return :  0.95%   |   
Eval Episode 1: Total Reward: 0.01, Portfolio Return: 0.95%, Market Return: -8.65%, Excess Return: 9.60%, Steps: 720
Market Return : -8.65%   |   Portfolio Return :  0.95%   |   
Eval Episode 2: Total Reward: 0.01, Portfolio Return: 0.95%, Market Return: -8.65%, Excess Return: 9.60%, Steps: 720
Market Return : -8.65%   |   Portfolio Return :  0.95%   |   
Eval Episode 3: Total Reward: 0.01, Portfolio Return: 0.95%, Market Return: -8.65%, Excess Return: 9.60%, Steps: 720
Market Return : -8.65%   |   Portfolio Return :  0.94%   |   
Eval Episode 4: Total Reward: 0.01, Portfolio Return: 0.94%, Market Return: -8.65%, Excess Return: 9.59%, Steps: 720
Market Return : -8.65%   |   Portfolio Return :  0.95%   |   
Eval Episode 5: Total Reward: 0.01, Portfolio Return: 0.95%, Market Return: -8.65%, Excess Return: 9.60%, Steps: 720
Market Return : -8.65%   |   Portfolio Return :  0.95%   |   


## 🎯 The `objective` Function: Our Optimization Target

This function defines a **single trial** for our Optuna hyperparameter search. Its job is to take a set of hyperparameters, train an agent, evaluate it, and return a single score that Optuna will try to maximize.

The entire process works in these steps:

1.  **Suggest Hyperparameters:** Optuna provides a `trial` object. We use this object to "suggest" a unique combination of hyperparameters (like learning rate, batch size, and network layers) from the search space we defined (e.g., `lr` between `1e-5` and `1e-3`). This set of parameters is stored in the `ppo_hps` dictionary.

2.  **Initialize Agent:** A *brand new* `PPOAgent` is created from scratch using the `ppo_hps` for this specific trial.

3.  **Train the Agent:** The new agent is trained for a fixed `TOTAL_TIMESTEPS` (500,000 steps) on our `env_train` (the main training dataset). This is a fast-but-thorough training run to see how well this set of hyperparameters can learn a policy.

4.  **Evaluate on Validation Set:** After training, the agent's performance is measured on the **`env_validation`** (our 3-month validation set). We specifically *do not* use our final `env_eval` test set here. This is critical to prevent data leakage and ensures our final test results are unbiased.

5.  **Return the Score:** We calculate the agent's average **`mean_excess_return`** (its portfolio return *minus* the market's return) on the validation set. This single number is the final score for the trial.

Optuna will run this function 50 times, each time generating a new set of hyperparameters and getting a new `mean_excess_return`. It will keep track of which hyperparameters produced the highest score.

In [157]:
def objective(trial):
    # --- 1. Hyperparameters ---
    ppo_hps = {
        "lr": trial.suggest_float("lr", 1e-5, 1e-3, log=True),
        "gamma": trial.suggest_categorical("gamma", [0.98, 0.99, 0.995, 0.999]),
        "gae_lambda": trial.suggest_float("gae_lambda", 0.8, 0.999),
        # Increase lower bound for stability and exploration
        "entropy_beta": trial.suggest_float("entropy_beta", 1e-3, 1e-2, log=True), 
        "clip_epsilon": trial.suggest_float("clip_epsilon", 0.1, 0.3),
        "ppo_epochs": trial.suggest_int("ppo_epochs", 5, 15), # Narrowed range slightly
        "batch_size": trial.suggest_categorical("batch_size", [64, 128, 256, 512]),
        "hidden_size": trial.suggest_categorical("hidden_size", [256, 512, 1024]), 
        "n_layers": trial.suggest_int("n_layers", 2, 4)
    }

    # --- 2. Setup ---
    TOTAL_TIMESTEPS = 200_00 
    ROLLOUT_STEPS = 2048
    PRUNING_INTERVAL_STEPS = 10 * ROLLOUT_STEPS # Evaluate every ~20k steps
    
    agent = PPOAgent(
        state_size=state_shape,
        n_actions=n_actions,
        **ppo_hps
    )

    # --- 3. Training Loop (Standard PPO Loop) ---
    obs, info = env_train.reset()
    
    for step in range(1, TOTAL_TIMESTEPS + 1):
        # ... PPO sampling and storing logic ...
        action, value, log_prob = agent.get_action_value_logprob(obs)
        next_obs, reward, done, truncated, info = env_train.step(action)
        agent.store(obs, action, reward, value, done, log_prob)
        obs = next_obs
        
        # Update Phase
        if step % ROLLOUT_STEPS == 0:
            next_value = 0.0
            if not done:
                with torch.no_grad():
                    _, next_value_tensor = agent.network(
                        torch.tensor(obs, dtype=torch.float32, device=agent.device).unsqueeze(0)
                    )
                    next_value = next_value_tensor.item()
            agent.update(next_value)

            # --- 4. PRUNING LOGIC (Using Mean Excess Return) ---
            if step % PRUNING_INTERVAL_STEPS == 0:
                val_results = evaluate_agent(agent, env_validation, num_episodes=5, render=False)
                
                # Metric for pruning: Mean Excess Return (Portfolio - Market)
                mean_excess_return = val_results['portfolio_return'].mean() - val_results['market_return'].mean()
                
                # Report to Optuna
                trial.report(mean_excess_return, step)

                # Handle Pruning
                if trial.should_prune():
                    raise optuna.TrialPruned()

        if done or truncated:
            obs, info = env_train.reset()

    # --- 5. Final Evaluation (Robust) ---
    eval_results = evaluate_agent(agent, env_validation, num_episodes=20, render=False)
    
    portfolio_returns = eval_results['portfolio_return']
    market_returns = eval_results['market_return']
    excess_returns = portfolio_returns - market_returns
    
    # --- 6. The Final Score: Maximize Mean Excess Return (Most Stable) ---
    final_score = excess_returns.mean()
    
    return final_score

## ⚙️ Hyperparameter Optimization with Optuna

Now that we have our agent, environment, and a proper validation setup, we need to find the best set of hyperparameters (like learning rate, network size, etc.) to make our agent as profitable as possible.

Doing this by hand is very time-consuming. Instead, we use the `Optuna` library to automate this search.

This code block sets up and runs the optimization study:

1.  **Sampler & Pruner:** We initialize a `TPESampler` (a smart algorithm to guess the next best parameters) and a `MedianPruner` (which automatically stops trials that are performing poorly to save time).
2.  **Create Study:** We create a `study` object and tell it we want to `"maximize"` the score.
3.  **Run Optimization:** We call `study.optimize(objective, n_trials=50)`. This will:
    * Call our `objective` function 50 times.
    * Each call, it passes in a new set of hyperparameters for the `trial`.
    * Our `objective` function trains a temporary agent on `env_train`, evaluates it on `env_validation`, and returns the **mean excess return**.
    * Optuna keeps track of which hyperparameters produced the highest excess return.
4.  **Get Results:** After the search is complete, the script prints the best score and stores the winning set of hyperparameters in the `best_hps` dictionary.

We will use this `best_hps` dictionary in the next step to train our single, final agent.

In [161]:
sampler = TPESampler(seed=42)
pruner = MedianPruner()

# Re-create the study to start fresh
study = optuna.create_study(
    study_name="ppo_trading_agent_optimization_v2",
    direction="maximize",
    sampler=sampler,
    pruner=pruner
)

# Start the optimization again (it should no longer fail)
try:
    study.optimize(objective, n_trials=50, timeout=5400) # 50 trials, 1h 30min timeout
except KeyboardInterrupt:
    print("Optimization stopped manually.")


Market Return : 76.67%   |   Portfolio Return : -53.83%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Market Return :  4.78%   |   Portfolio Return : -11.65%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Market Return :  4.78%   |   Portfolio Return : -11.65%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Market Return :  4.78%   |   Portfolio Return : -11.64%

[I 2025-11-19 00:47:00,860] Trial 0 finished with value: -0.16421500000000006 and parameters: {'lr': 5.6115164153345e-05, 'gamma': 0.98, 'gae_lambda': 0.8310429095469044, 'entropy_beta': 0.001143098387631322, 'clip_epsilon': 0.27323522915498705, 'ppo_epochs': 11, 'batch_size': 256, 'hidden_size': 256, 'n_layers': 2}. Best is trial 0 with value: -0.16421500000000006.


Market Return :  4.78%   |   Portfolio Return : -11.64%   |   
Saved evaluation results to evaluation_results.csv
Market Return : 76.67%   |   Portfolio Return : -52.77%   |   
Market Return : 76.67%   |   Portfolio Return : -10.08%   |   
Market Return : 76.67%   |   Portfolio Return : -25.33%   |   
Market Return :  4.78%   |   Portfolio Return : -10.20%   |   
Market Return :  4.78%   |   Portfolio Return : -10.20%   |   
Market Return :  4.78%   |   Portfolio Return : -10.20%   |   
Market Return :  4.78%   |   Portfolio Return : -10.20%   |   
Market Return :  4.78%   |   Portfolio Return : -10.20%   |   
Market Return :  4.78%   |   Portfolio Return : -10.20%   |   
Market Return :  4.78%   |   Portfolio Return : -10.19%   |   
Market Return :  4.78%   |   Portfolio Return : -10.19%   |   
Market Return :  4.78%   |   Portfolio Return : -10.19%   |   
Market Return :  4.78%   |   Portfolio Return : -10.20%   |   
Market Return :  4.78%   |   Portfolio Return : -10.19%   |   
Mark

[I 2025-11-19 00:48:58,798] Trial 1 finished with value: -0.14976499999999998 and parameters: {'lr': 0.00011207606211860574, 'gamma': 0.995, 'gae_lambda': 0.8581367850585084, 'entropy_beta': 0.0023246728489504354, 'clip_epsilon': 0.19121399684340717, 'ppo_epochs': 13, 'batch_size': 256, 'hidden_size': 256, 'n_layers': 4}. Best is trial 1 with value: -0.14976499999999998.


Market Return :  4.78%   |   Portfolio Return : -10.20%   |   
Saved evaluation results to evaluation_results.csv
Market Return : 76.67%   |   Portfolio Return : -33.90%   |   
Market Return : 76.67%   |   Portfolio Return : -42.84%   |   
Market Return : 76.67%   |   Portfolio Return : -31.84%   |   
Market Return :  4.78%   |   Portfolio Return : -7.95%   |   
Market Return :  4.78%   |   Portfolio Return : -6.96%   |   
Market Return :  4.78%   |   Portfolio Return : -6.96%   |   
Market Return :  4.78%   |   Portfolio Return : -6.96%   |   
Market Return :  4.78%   |   Portfolio Return : -6.95%   |   
Market Return :  4.78%   |   Portfolio Return : -6.95%   |   
Market Return :  4.78%   |   Portfolio Return : -6.96%   |   
Market Return :  4.78%   |   Portfolio Return : -7.27%   |   
Market Return :  4.78%   |   Portfolio Return : -6.95%   |   
Market Return :  4.78%   |   Portfolio Return : -7.95%   |   
Market Return :  4.78%   |   Portfolio Return : -6.96%   |   
Market Return :

...

Market Return :  4.78%   |   Portfolio Return : -3.52%   |   
Saved evaluation results to evaluation_results.csv
Market Return : 76.67%   |   Portfolio Return : -43.84%   |   
Market Return : 76.67%   |   Portfolio Return : -11.08%   |   
Market Return : 76.67%   |   Portfolio Return : -53.27%   |   
Market Return :  4.78%   |   Portfolio Return :  3.98%   |   
Market Return :  4.78%   |   Portfolio Return :  3.98%   |   
Market Return :  4.78%   |   Portfolio Return :  3.97%   |   
Market Return :  4.78%   |   Portfolio Return :  3.97%   |   
Market Return :  4.78%   |   Portfolio Return :  3.98%   |   
Market Return :  4.78%   |   Portfolio Return :  3.98%   |   
Market Return :  4.78%   |   Portfolio Return :  3.98%   |   
Market Return :  4.78%   |   Portfolio Return :  3.97%   |   
Market Return :  4.78%   |   Portfolio Return :  3.97%   |   
Market Return :  4.78%   |   Portfolio Return :  3.97%   |   
Market Return :  4.78%   |   Portfolio Return :  3.98%   |   
Market Return : 

[I 2025-11-19 02:09:35,291] Trial 21 finished with value: -0.008045000000000002 and parameters: {'lr': 1.0007236731812058e-05, 'gamma': 0.99, 'gae_lambda': 0.9085127260663897, 'entropy_beta': 0.0010874410132689727, 'clip_epsilon': 0.29468582820569833, 'ppo_epochs': 5, 'batch_size': 64, 'hidden_size': 1024, 'n_layers': 4}. Best is trial 6 with value: 0.02615.


Market Return :  4.78%   |   Portfolio Return :  3.97%   |   
Saved evaluation results to evaluation_results.csv
Market Return : 76.67%   |   Portfolio Return : -45.26%   |   
Market Return : 76.67%   |   Portfolio Return : -60.36%   |   
Market Return : 76.67%   |   Portfolio Return : -57.74%   |   
Market Return :  4.78%   |   Portfolio Return :  5.67%   |   
Market Return :  4.78%   |   Portfolio Return :  5.67%   |   
Market Return :  4.78%   |   Portfolio Return :  5.68%   |   
Market Return :  4.78%   |   Portfolio Return :  5.67%   |   
Market Return :  4.78%   |   Portfolio Return :  5.67%   |   
Market Return :  4.78%   |   Portfolio Return :  5.67%   |   
Market Return :  4.78%   |   Portfolio Return :  5.67%   |   
Market Return :  4.78%   |   Portfolio Return :  5.67%   |   
Market Return :  4.78%   |   Portfolio Return :  5.68%   |   
Market Return :  4.78%   |   Portfolio Return :  5.67%   |   
Market Return :  4.78%   |   Portfolio Return :  5.67%   |   
Market Return : 

[I 2025-11-19 03:02:12,626] Trial 22 finished with value: 0.008929999999999997 and parameters: {'lr': 1.4761920471576542e-05, 'gamma': 0.99, 'gae_lambda': 0.9763547200275244, 'entropy_beta': 0.0013934014432135133, 'clip_epsilon': 0.2909928407015177, 'ppo_epochs': 6, 'batch_size': 64, 'hidden_size': 1024, 'n_layers': 4}. Best is trial 6 with value: 0.02615.


Market Return :  4.78%   |   Portfolio Return :  5.67%   |   
Saved evaluation results to evaluation_results.csv


In [162]:
# --- Print Results ---
print("\n--- Optimization Finished ---")
print(f"Number of finished trials: {len(study.trials)}")

print("\nBest trial:")
best_trial = study.best_trial
print(f"Value (Mean Excess Return): {best_trial.value:.4f}")

print("Best Hyperparameters:")
for key, value in best_trial.params.items():
    print(f"{key}: {value}")

# You can now use these best hyperparameters to train your final agent
# for a longer duration (e.g., more TOTAL_TIMESTEPS).
best_hps = best_trial.params
print("\nBest hyperparameters dictionary:")
print(best_hps)


--- Optimization Finished ---
Number of finished trials: 23

Best trial:
Value (Mean Excess Return): 0.0261
Best Hyperparameters:
lr: 4.253162363790868e-05
gamma: 0.99
gae_lambda: 0.9503546765700667
entropy_beta: 0.0016935505549297925
clip_epsilon: 0.1153959819657586
ppo_epochs: 8
batch_size: 128
hidden_size: 256
n_layers: 4

Best hyperparameters dictionary:
{'lr': 4.253162363790868e-05, 'gamma': 0.99, 'gae_lambda': 0.9503546765700667, 'entropy_beta': 0.0016935505549297925, 'clip_epsilon': 0.1153959819657586, 'ppo_epochs': 8, 'batch_size': 128, 'hidden_size': 256, 'n_layers': 4}


--- Optimization Finished ---
Number of finished trials: 16

- Best trial:
- Value (Mean Excess Return): 0.0331
- Best Hyperparameters:
- lr: 2.0738922307588755e-05
- gamma: 0.999
- gae_lambda: 0.9455630627411082
- entropy_beta: 0.007109875990561107
- clip_epsilon: 0.22401267536164615
- ppo_epochs: 13
- batch_size: 512
- hidden_size: 1024
- n_layers: 2

- Best hyperparameters dictionary:
- {'lr': 2.0738922307588755e-05, 'gamma': 0.999, 'gae_lambda': 0.9455630627411082, 'entropy_beta': 0.007109875990561107, 'clip_epsilon': 0.22401267536164615, 'ppo_epochs': 13, 'batch_size': 512, 'hidden_size': 1024, 'n_layers': 2}

## 🚀 Training the Final Agent

This is the main training script. After identifying the best hyperparameters with Optuna (`best_hps`), we now use them to train a single, "production-ready" agent.

This process is designed to be robust and follows standard machine learning practices:

1.  **Consolidate Training Data:** We combine our original `df_train` and `df_validation` sets into one large `df_train_full`. This gives our final agent the maximum amount of data (the full year) to learn from.
2.  **Initialize Optimal Agent:** We create a new `PPOAgent` instance using the exact `best_hps` (learning rate, hidden size, etc.) discovered by Optuna.
3.  **Train on Full Data:** The agent is trained for a long duration (1 million steps) on the `env_train_full`.
4.  **Validate and Save:** Critically, every 5 updates, the agent is evaluated against the **`env_validation`** (the 3-month validation set).
5.  **Save Best Model:** We save the agent's network weights *only if* its **mean excess return** (its performance *above* the market) on the validation set improves. This ensures we are not just saving a lucky model, but the one that is most generalizable and best at *beating* the market.

The final saved file, `models/ppo_trading_agent.pth`, will contain the weights of the single best-performing model, ready for the final, unseen test.

In [163]:
# --- 1. Create a "full training" environment ---
# This combines your original train and validation sets
df_train_full = df.loc['2024-10-01':'2025-09-30'] 

env_train_full = gym.make("TradingEnv",
        name= "BTCUSD_Eval",
        df = df_train_full, 
        windows=WINDOW_SIZE,
        positions = POSITIONS,
        trading_fees = TRADING_FEES, 
        borrow_interest_rate= BORROW_INTEREST_RATE,
        reward_function=custom_reward
    )

# --- 2. Get environment parameters ---
state_shape = env_train_full.observation_space.shape  # Use the new env
n_actions = env_train_full.action_space.n

print(f"State shape: {state_shape}")
print(f"Number of actions: {n_actions}")

# --- 3. Training hyperparameters ---
TOTAL_TIMESTEPS = 1_000_00     
ROLLOUT_STEPS = 2048         
EVAL_EVERY_N_UPDATES = 5     
MODEL_SAVE_PATH = "models/ppo_trading_agent_v2.pth"

# --- 4. Initialize agent with best hyperparameters ---
agent = PPOAgent(state_size=state_shape, n_actions=n_actions, **best_hps)

# --- 5. Training & logging setup ---
all_episode_rewards = [] 
episode_rewards = []     
best_eval_excess_return = -float('inf') # <--- We track BEST EXCESS RETURN
update_count = 0

print(f"Starting training for {TOTAL_TIMESTEPS} timesteps...")
print(f"Will update every {ROLLOUT_STEPS} steps.")
print(f"Evaluating every {EVAL_EVERY_N_UPDATES} updates.")

# --- 6. Main training loop ---
obs, info = env_train_full.reset() # <--- Use the full training env

for step in range(1, TOTAL_TIMESTEPS + 1):
    action, value, log_prob = agent.get_action_value_logprob(obs)
    
    # --- Use the full training env ---
    next_obs, reward, done, truncated, info = env_train_full.step(action) 
    
    agent.store(obs, action, reward, value, done, log_prob)
    episode_rewards.append(reward)
    obs = next_obs
    
    if step % ROLLOUT_STEPS == 0:
        update_count += 1
        
        next_value = 0.0
        if not done:
            with torch.no_grad():
                _, next_value_tensor = agent.network(torch.tensor(obs, dtype=torch.float32, device=agent.device).unsqueeze(0))
                next_value = next_value_tensor.item()
        
        losses = agent.update(next_value)
        
        print(f"\nUpdate {update_count} (Step {step}/{TOTAL_TIMESTEPS})")
        print(f"  Actor Loss: {losses['actor_loss']:.4f}, Critic Loss: {losses['critic_loss']:.4f}")
        if len(all_episode_rewards) > 0:
            print(f"  Mean Reward (last 10 ep): {np.mean(all_episode_rewards[-10:]):.4f}")
        
        # --- Periodic evaluation on the VALIDATION set ---
        if update_count % EVAL_EVERY_N_UPDATES == 0:
            print("--- Running Validation ---")
            
            # --- FIX 1: Evaluate on env_validation ---
            eval_results = evaluate_agent(agent, env_validation, num_episodes=20, render=False)
            
            mean_eval_return = eval_results['portfolio_return'].mean()
            market_return = eval_results['market_return'].mean()
            
            # --- FIX 2: Calculate and save based on EXCESS return ---
            mean_excess_return = mean_eval_return - market_return 
            
            print(f"  Mean Validation Portfolio Return: {mean_eval_return:.2%}")
            print(f"  Mean Validation Excess Return: {mean_excess_return:.2%}")
            print(f"  Market Return: {market_return:.2%}")
            
            if mean_excess_return > best_eval_excess_return:
                best_eval_excess_return = mean_excess_return
                torch.save(agent.network.state_dict(), MODEL_SAVE_PATH)
                print(f"  *** New best model saved with EXCESS return {best_eval_excess_return:.2%} ***")
            print("--------------------------")
            
    if done or truncated:
        all_episode_rewards.append(sum(episode_rewards))
        episode_rewards = []
        obs, info = env_train_full.reset() # <--- Use the full training env

print("\nTraining finished.")
print(f"Best model saved to {MODEL_SAVE_PATH} with validation excess return: {best_eval_excess_return:.2%}")

State shape: (48, 19)
Number of actions: 9
Starting training for 100000 timesteps...
Will update every 2048 steps.
Evaluating every 5 updates.

Update 1 (Step 2048/100000)
  Actor Loss: -0.0046, Critic Loss: 0.0001

Update 2 (Step 4096/100000)
  Actor Loss: -0.0074, Critic Loss: 0.0003

Update 3 (Step 6144/100000)
  Actor Loss: -0.0058, Critic Loss: 0.0001

Update 4 (Step 8192/100000)
  Actor Loss: -0.0075, Critic Loss: 0.0002
Market Return : 88.05%   |   Portfolio Return : -49.54%   |   

Update 5 (Step 10240/100000)
  Actor Loss: -0.0059, Critic Loss: 0.0002
  Mean Reward (last 10 ep): 0.0135
--- Running Validation ---
Market Return :  4.78%   |   Portfolio Return :  3.32%   |   
Market Return :  4.78%   |   Portfolio Return :  3.33%   |   
Market Return :  4.78%   |   Portfolio Return :  3.33%   |   
Market Return :  4.78%   |   Portfolio Return :  3.33%   |   
Market Return :  4.78%   |   Portfolio Return :  3.32%   |   
Market Return :  4.78%   |   Portfolio Return :  3.33%   |   

# 🏁 Final Model Evaluation on Unseen Test Data
This code block performs the final and most important test of the project.

After completing the Optuna hyperparameter search, we have the best_hps dictionary, which defines the optimal agent architecture. Our main training loop has saved the best-performing model weights (based on validation performance) to models/ppo_trading_agent.pth.

This final script does the following:

1. Initialize Optimal Agent: It creates a new PPOAgent instance using the exact architecture and hyperparameters found by Optuna (best_hps).

2. Load Trained Weights: It loads the saved weights from models/ppo_trading_agent.pth into this new agent.

3. Set to Evaluation Mode: It calls trained_agent.network.eval(). This is a standard PyTorch step that turns off training-specific behaviors (like dropout) to ensure the model's output is deterministic.

4. Run on Test Set: It calls the evaluate_agent function on env_eval. This is the first and only time our agent encounters the test data. This gives us a fair, unbiased assessment of how well it generalized.

5. Save & Report: The final performance metrics are saved to evaluation_results.csv (as required by the project) and printed. This result demonstrates the agent's final performance against the market.

In [ ]:
# --- 1. Initialize agent with the BEST hyperparameters from Optuna ---
# (best_hps should be the dictionary you got from your Optuna study)
print(f"Loading best model with hyperparameters: {best_hps}")
trained_agent = PPOAgent(
    state_size=state_shape, 
    n_actions=n_actions,
    **best_hps  
)

# --- 2. Load the saved model weights ---
MODEL_SAVE_PATH = "models/ppo_trading_agent_v2.pth"
trained_agent.network.load_state_dict(torch.load(MODEL_SAVE_PATH))

# Set the network to evaluation mode (this is correct)
trained_agent.network.eval()

print("--- Evaluating final trained agent on TEST SET ---")

# --- 3. Evaluate the agent on the unseen TEST set ---
df_results = evaluate_agent(
    trained_agent,
    env_validation,  
    num_episodes=20,
    render=True,
    csv_path=eval_folder / "evaluation_results.csv",
    renderer_logs_dir=eval_folder / "render_logs"
)

print("--- Final Evaluation Complete ---")
print(df_results)

Loading best model with hyperparameters: {'lr': 4.253162363790868e-05, 'gamma': 0.99, 'gae_lambda': 0.9503546765700667, 'entropy_beta': 0.0016935505549297925, 'clip_epsilon': 0.1153959819657586, 'ppo_epochs': 8, 'batch_size': 128, 'hidden_size': 256, 'n_layers': 4}
--- Evaluating final trained agent on TEST SET ---
Market Return :  4.78%   |   Portfolio Return :  8.63%   |   
Eval Episode 1: Total Reward: 0.11, Portfolio Return: 8.63%, Market Return: 4.78%, Excess Return: 3.85%, Steps: 2160
Market Return :  4.78%   |   Portfolio Return :  8.62%   |   
Eval Episode 2: Total Reward: 0.11, Portfolio Return: 8.62%, Market Return: 4.78%, Excess Return: 3.84%, Steps: 2160
Market Return :  4.78%   |   Portfolio Return :  8.64%   |   
Eval Episode 3: Total Reward: 0.11, Portfolio Return: 8.64%, Market Return: 4.78%, Excess Return: 3.86%, Steps: 2160
Market Return :  4.78%   |   Portfolio Return :  8.64%   |   
Eval Episode 4: Total Reward: 0.11, Portfolio Return: 8.64%, Market Return: 4.78%, E

In [166]:
df_results

,episode,portfolio_return,market_return,excess_return,steps,total_reward
0,1,0.0863,0.0478,0.0385,2160,0.105244
1,2,0.0862,0.0478,0.0384,2160,0.105244
2,3,0.0864,0.0478,0.0386,2160,0.105244
3,4,0.0864,0.0478,0.0386,2160,0.105244
4,5,0.0862,0.0478,0.0384,2160,0.105244
5,6,0.0862,0.0478,0.0384,2160,0.105244
6,7,0.0864,0.0478,0.0386,2160,0.105244
7,8,0.0862,0.0478,0.0384,2160,0.105244
8,9,0.0862,0.0478,0.0384,2160,0.105244
9,10,0.0862,0.0478,0.0384,2160,0.105244


## 📈 Conclusion: PPO Trading Agent Performance Analysis

The Reinforcement Learning project successfully implemented and trained a PPO-based trading agent on **BTC/USDT** hourly data. The final evaluation on the unseen test set demonstrates that the agent developed a profitable strategy that **significantly outperformed the market baseline**.

---

### 1. 🎯 Final Agent Performance (Test Set)

The critical measure of success for a trading agent is the **Excess Return**, which is the portfolio return minus the market's return over the same period.

| Metric | Random Agent (Baseline) | PPO Agent (Final Model) |
| :--- | :---: | :---: |
| **Market Return** (BTC/USDT) | -8.65% | **4.78%** |
| **Portfolio Return** (Mean) | Varies (e.g., -6.70% to 7.25%) | **8.63%** |
| **Excess Return** (Mean) | Varies (e.g., -10.73% to 15.90%) | **3.85%** |

* **Market Return Discrepancy:** Note that the Market Return printed for the Random Agent is **-8.65%**, while for the Final Agent's evaluation it is **4.78%**. This suggests the Random Agent was incorrectly evaluated on the training/validation data, **not the final test set**, as the final evaluation consistently reports a **4.78%** market return (from October 2025 data slice).
* **Result Interpretation:** Based on the **Final Evaluation**, the market (BTC/USDT) returned an average of **+4.78%** over the test period. The PPO agent achieved an average **Portfolio Return of +8.63%**, resulting in a robust **Excess Return of +3.85%**. This is a solid, positive result, indicating the agent learned a policy that successfully generated alpha (return above the benchmark).

---

### 2. ⚙️ Optimal Hyperparameters

The **Optuna** hyperparameter search identified the following optimal set of parameters (from Trial 6) which were used to train the final agent:

* **Learning Rate (`lr`):** $4.25 \times 10^{-5}$
* **Gamma (`gamma`):** 0.99
* **GAE Lambda (`gae_lambda`):** 0.950
* **Entropy Beta (`entropy_beta`):** $1.69 \times 10^{-3}$
* **PPO Epochs (`ppo_epochs`):** 8
* **Batch Size (`batch_size`):** 128
* **Network Size (`hidden_size`):** 256
* **Network Layers (`n_layers`):** 4

The low learning rate and the gamma value close to 1 are typical for financial time series, suggesting the model needs a **long-term view** (high gamma) and **stable, small updates** (low LR) to navigate market complexities.

---

### 3. 🧠 Agent Architecture & Training Success

* **PPO for Stability:** The choice of **Proximal Policy Optimization (PPO)** proved effective, as it is known for its stability. The successful non-negative excess return suggests that the clipping mechanism and value estimation successfully stabilized the learning process. * **CNN-Based Feature Extraction:** The use of the `ActorCriticNetwork` incorporating **Convolutional Neural Network (CNN)** layers to process the time-series data window (size 48) was crucial. This architecture allowed the agent to extract spatial and temporal patterns from the features (like RSI, MACD, Sharpe Ratio, etc.) over the 48-hour window, which is likely key to its superior performance over the random baseline.
* **Custom Reward Function:** The implementation of the `custom_reward` function, which included a subtle **neutrality penalty**, successfully mitigated the initial tendency of the agent to stay passive. This penalty incentivized the agent to take long or short positions when the projected gain outweighed the small risk of being wrong, leading to a more active and profitable policy.

In conclusion, the PPO-powered trading agent achieved its goal by successfully developing a policy that generated **3.85% alpha** over the market benchmark on the unseen test data.